In [16]:
## import pandas as pd
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import numpy as np 
import itertools
import tensorflow.keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from tensorflow.keras.models import Sequential 
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPool2D 
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten
from tensorflow.keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import efficientnet.keras as enet
import keras_applications as app
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Model
import keras
#%matplotlib inline
import math 
import datetime
import time
CUDA_VISIBLE_DEVICES = 0
os.environ['CUDA_VISIBLE_DEVICES'] = '0'



In [2]:

#Default dimensions we found online
img_width, img_height = 224, 224 
 
#Create a bottleneck file
top_model_weights_path = "bottleneck_fc_model_50_50.h5"
# loading up our datasets
train_data_dir = '/media/hades/DISK 1 - Drive 2/Fish/Test7/Train' 
validation_data_dir = '/media/hades/DISK 1 - Drive 2/Fish/Test7/Validate' 
test_data_dir = '/media/hades/DISK 1 - Drive 2/Fish/Test7/Test'
 
# number of epochs to train top model 
epochs = 50 #this has been changed after multiple model run 
# batch size used by flow_from_directory and predict_generator 
batch_size = 50 


In [3]:
import efficientnet.keras as enet

#Loading vgc16 model
print("HAS")
#vgg16 = app.efficientnet.EfficientNetB7(utils = tensorflow.keras.utils, models=tensorflow.keras.models, layers = tensorflow.keras.layers, backend = tensorflow.keras.backend, include_top=False, input_shape=(224,224,3), pooling='avg', weights='imagenet')
#vgg16 = app.vgg16.VGG16(utils = keras.utils, models=keras.models, layers = keras.layers, backend = keras.backend, include_top=False, input_shape=(224,224,3), pooling='avg', weights='imagenet')
#vgg16 = applications.VGG16(include_top=False, weights=’imagenet’)
#vgg16 = app.mobilenet_v3.MobileNetV3(utils = keras.utils, models=keras.models, layers = keras.layers, backend = keras.backend, include_top=False, input_shape=(224,224,3), pooling='avg', weights='imagenet')
#vgg16 = app.resnet50.ResNet50(utils = keras.utils, models=keras.models, layers = keras.layers, backend = keras.backend, include_top=False, input_shape=(224,224,3), pooling='avg', weights='imagenet')
vgg16=app.mobilenet.MobileNet(utils = tensorflow.keras.utils, models=tensorflow.keras.models, layers = tensorflow.keras.layers, backend = tensorflow.keras.backend, include_top=True, input_shape=(224,224,3), pooling='avg', weights='imagenet')
datagen = ImageDataGenerator(rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
# datagen = ImageDataGenerator(rescale=1. / 255,
#       )
print("HADOH")
#needed to create the bottleneck .npy files

HAS
HADOH


In [4]:
#from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model



#plot_model(model, to_file='toplayers.png', show_shapes=True, show_layer_names=True)
vgg16.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [5]:
#__this can take an hour and half to run so only run it once. 
#once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__
start = datetime.datetime.now()
print("ASD1")
generator = datagen.flow_from_directory( 
    train_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode=None, 
    shuffle=False) 
 
nb_train_samples = len(generator.filenames) 
num_classes = len(generator.class_indices) 
 
predict_size_train = int(math.ceil(nb_train_samples / batch_size)) 
 
bottleneck_features_train = vgg16.predict_generator(generator, predict_size_train) 
vgg16.LOAD_TRUNCATED_IMAGES = True

np.save('bottleneck_features_train_50_50.npy', bottleneck_features_train)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

ASD1
Found 5150 images belonging to 55 classes.


/home/hades/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
/home/hades/.local/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Time:  0:02:15.559115


In [6]:
#__this can take an hour and half to run so only run it once. 
#once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__
start = datetime.datetime.now()
 
generator = datagen.flow_from_directory( 
    test_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode=None, 
    shuffle=False) 
 
nb_validation_samples = len(generator.filenames) 
num_classes = len(generator.class_indices) 
 
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size)) 
 
bottleneck_features_validation = vgg16.predict_generator(generator, predict_size_validation) 
 
np.save('bottleneck_features_validation_50_50.npy', bottleneck_features_validation)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

Found 537 images belonging to 55 classes.


/home/hades/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Time:  0:00:14.431992


In [7]:
#training data
generator_top = datagen.flow_from_directory( 
   train_data_dir, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_train_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
train_data = np.load('bottleneck_features_train_50_50.npy') 
 
# get the class labels for the training data, in the original order 
train_labels = generator_top.classes 
 
# convert the training labels to categorical vectors 
train_labels = to_categorical(train_labels, num_classes=num_classes)


Found 5150 images belonging to 55 classes.


In [8]:
#training data
generator_top = datagen.flow_from_directory( 
   test_data_dir, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_validation_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
validation_data = np.load('bottleneck_features_validation_50_50.npy') 
 
# get the class labels for the training data, in the original order 
validation_labels = generator_top.classes 
 
# convert the training labels to categorical vectors 
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

#validation_data = validation_data.reshape((-1, 224, 224, 3))

Found 537 images belonging to 55 classes.


In [9]:
list_file=generator_top.filenames

In [10]:
# Swish defination
from tensorflow.keras.backend import sigmoid

class SwishActivation(Activation):
    
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from tensorflow.keras.layers import Activation
get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

In [11]:
#This is the best model we found. For additional models, check out I_notebook.ipynb
from tensorflow.keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPool2D 
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Dropout,Dense,Flatten,BatchNormalization,Conv2D
start = datetime.datetime.now()
#print(train_data[2].shape[1])
# model = Sequential() 
# #model.add(Flatten(input_shape=train_data.shape[1:])) 
# model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.5)) 
# model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.3)) 
# model.add(Dense(num_classes, activation='softmax'))
# model.compile(loss='categorical_crossentropy',
#    optimizer=optimizers.Adam(lr=1e-4),
#    metrics=['acc'])

# model.add(Dense(200, activation=keras.layers.LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.5)) 
# model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.3))
# model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.3))
# model.add(Dense(num_classes, activation='softmax'))
# model.compile(loss='categorical_crossentropy',
#    optimizer=optimizers.Adam(lr=1e-4),
# #    metrics=['acc'])

###########


###################################################################################
# model = Sequential() 
# model.add(BatchNormalization()) 
# model.add(Dropout(0.7)) 
# model.add(Dense(512)) 
# model.add(BatchNormalization())
# model.add(Activation(swish_act))

# model.add(Dense(num_classes, activation="softmax"))

# model.compile(loss='categorical_crossentropy',
#    optimizer=optimizers.Adam(lr=1e-4),
#    metrics=['acc'])
######################################################################



model = Sequential()
# model.add(tensorflow.keras.layers.Dense(1024, activation='relu'))
# model.add(tensorflow.keras.layers.Dense(1024, activation='relu'))
# model.add(tensorflow.keras.layers.Dense(512, activation='relu'))

#model.add(BatchNormalization()) 
#model.add(Dropout(0.7)) 
#model.add(Dense(512)) 
#model.add(BatchNormalization())
#model.add(Activation(swish_act))
#model.add(Dropout(0.5))
#model.add(Dense(128)) 
#model.add(BatchNormalization())
#model.add(Activation(swish_act))
model.add(Dense(num_classes, activation="softmax"))

model.compile(loss='categorical_crossentropy',
   optimizer=optimizers.Adam(lr=1e-4),
   metrics=['acc'])



# model = Sequential()

# # building 2 fully connected layer 
# #x = model.output

# x = BatchNormalization()(x)
# x = Dropout(0.7)(x)

# x = Dense(512)(x)
# x = BatchNormalization()(x)
# x = Activation(swish_act)(x)
# x = Dropout(0.5)(x)

# x = Dense(128)(x)
# x = BatchNormalization()(x)
# x = Activation(swish_act)(x)

# output layer
#predictions = Dense(10, activation="softmax")(x)

#model_final = Model(inputs = model.input, outputs = predictions)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(0.0001),
              metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, verbose=1,)
mcp_save = ModelCheckpoint('EnetB0_CIFAR10_TL.h5', save_best_only=True, monitor='val_acc')

history = model.fit(train_data, train_labels, 
   epochs=200,
   batch_size=batch_size, 
   validation_data=(validation_data, validation_labels))

model.save_weights(top_model_weights_path)
(eval_loss, eval_accuracy) = model.evaluate( 
    validation_data, validation_labels, batch_size=batch_size,     verbose=1)
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print('[INFO] Loss: {}'.format(eval_loss)) 
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

Epoch 1/200
103/103 [==============================] - 1s 4ms/step - loss: 3.9972 - accuracy: 0.0832 - val_loss: 3.9893 - val_accuracy: 0.1006
Epoch 2/200
103/103 [==============================] - 0s 1ms/step - loss: 3.9857 - accuracy: 0.1193 - val_loss: 3.9784 - val_accuracy: 0.1415
Epoch 3/200
103/103 [==============================] - 0s 1ms/step - loss: 3.9735 - accuracy: 0.1556 - val_loss: 3.9677 - val_accuracy: 0.1769
Epoch 4/200
103/103 [==============================] - 0s 1ms/step - loss: 3.9639 - accuracy: 0.1750 - val_loss: 3.9570 - val_accuracy: 0.1825
Epoch 5/200
103/103 [==============================] - 0s 1ms/step - loss: 3.9530 - accuracy: 0.1721 - val_loss: 3.9464 - val_accuracy: 0.1918
Epoch 6/200
103/103 [==============================] - 0s 2ms/step - loss: 3.9421 - accuracy: 0.1788 - val_loss: 3.9360 - val_accuracy: 0.1974
Epoch 7/200
103/103 [==============================] - 0s 1ms/step - loss: 3.9292 - accuracy: 0.1866 - val_loss: 3.9255 - val_accuracy: 0.2142

Epoch 58/200
103/103 [==============================] - 0s 1ms/step - loss: 3.4810 - accuracy: 0.2460 - val_loss: 3.4961 - val_accuracy: 0.2700
Epoch 59/200
103/103 [==============================] - 0s 1ms/step - loss: 3.4728 - accuracy: 0.2465 - val_loss: 3.4893 - val_accuracy: 0.2700
Epoch 60/200
103/103 [==============================] - 0s 2ms/step - loss: 3.4649 - accuracy: 0.2472 - val_loss: 3.4826 - val_accuracy: 0.2700
Epoch 61/200
103/103 [==============================] - 0s 1ms/step - loss: 3.4514 - accuracy: 0.2548 - val_loss: 3.4758 - val_accuracy: 0.2719
Epoch 62/200
103/103 [==============================] - 0s 2ms/step - loss: 3.4417 - accuracy: 0.2522 - val_loss: 3.4692 - val_accuracy: 0.2737
Epoch 63/200
103/103 [==============================] - 0s 2ms/step - loss: 3.4400 - accuracy: 0.2608 - val_loss: 3.4626 - val_accuracy: 0.2737
Epoch 64/200
103/103 [==============================] - 0s 2ms/step - loss: 3.4352 - accuracy: 0.2598 - val_loss: 3.4561 - val_accuracy:

Epoch 115/200
103/103 [==============================] - 0s 2ms/step - loss: 3.1383 - accuracy: 0.2694 - val_loss: 3.1757 - val_accuracy: 0.2905
Epoch 116/200
103/103 [==============================] - 0s 2ms/step - loss: 3.1449 - accuracy: 0.2624 - val_loss: 3.1710 - val_accuracy: 0.2905
Epoch 117/200
103/103 [==============================] - 0s 1ms/step - loss: 3.1465 - accuracy: 0.2561 - val_loss: 3.1664 - val_accuracy: 0.2905
Epoch 118/200
103/103 [==============================] - 0s 1ms/step - loss: 3.1452 - accuracy: 0.2604 - val_loss: 3.1618 - val_accuracy: 0.2942
Epoch 119/200
103/103 [==============================] - 0s 1ms/step - loss: 3.1281 - accuracy: 0.2643 - val_loss: 3.1572 - val_accuracy: 0.2961
Epoch 120/200
103/103 [==============================] - 0s 2ms/step - loss: 3.1340 - accuracy: 0.2656 - val_loss: 3.1527 - val_accuracy: 0.2998
Epoch 121/200
103/103 [==============================] - 0s 1ms/step - loss: 3.1011 - accuracy: 0.2689 - val_loss: 3.1481 - val_ac

103/103 [==============================] - 0s 1ms/step - loss: 2.9250 - accuracy: 0.2880 - val_loss: 2.9458 - val_accuracy: 0.3147
Epoch 172/200
103/103 [==============================] - 0s 1ms/step - loss: 2.9119 - accuracy: 0.2881 - val_loss: 2.9422 - val_accuracy: 0.3147
Epoch 173/200
103/103 [==============================] - 0s 1ms/step - loss: 2.9294 - accuracy: 0.2910 - val_loss: 2.9386 - val_accuracy: 0.3147
Epoch 174/200
103/103 [==============================] - 0s 1ms/step - loss: 2.9383 - accuracy: 0.2827 - val_loss: 2.9350 - val_accuracy: 0.3147
Epoch 175/200
103/103 [==============================] - 0s 1ms/step - loss: 2.9312 - accuracy: 0.2794 - val_loss: 2.9314 - val_accuracy: 0.3147
Epoch 176/200
103/103 [==============================] - 0s 1ms/step - loss: 2.9206 - accuracy: 0.2794 - val_loss: 2.9278 - val_accuracy: 0.3147
Epoch 177/200
103/103 [==============================] - 0s 2ms/step - loss: 2.9087 - accuracy: 0.2917 - val_loss: 2.9243 - val_accuracy: 0.3147

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 55)                55055     
Total params: 55,055
Trainable params: 55,055
Non-trainable params: 0
_________________________________________________________________


In [13]:

history = model.fit(train_data, train_labels, 
   epochs=10,
   batch_size=batch_size, 
   validation_data=(validation_data, validation_labels))

model.save_weights(top_model_weights_path)
(eval_loss, eval_accuracy) = model.evaluate( 
    validation_data, validation_labels, batch_size=batch_size,     verbose=1)
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print('[INFO] Loss: {}'.format(eval_loss)) 
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

Epoch 1/10
103/103 [==============================] - 0s 2ms/step - loss: 2.8331 - accuracy: 0.2934 - val_loss: 2.8430 - val_accuracy: 0.3147
Epoch 2/10
103/103 [==============================] - 0s 1ms/step - loss: 2.8301 - accuracy: 0.2932 - val_loss: 2.8398 - val_accuracy: 0.3147
Epoch 3/10
103/103 [==============================] - 0s 1ms/step - loss: 2.8271 - accuracy: 0.2940 - val_loss: 2.8366 - val_accuracy: 0.3147
Epoch 4/10
103/103 [==============================] - 0s 1ms/step - loss: 2.8242 - accuracy: 0.2938 - val_loss: 2.8334 - val_accuracy: 0.3147
Epoch 5/10
103/103 [==============================] - 0s 1ms/step - loss: 2.8213 - accuracy: 0.2942 - val_loss: 2.8302 - val_accuracy: 0.3147
Epoch 6/10
103/103 [==============================] - 0s 1ms/step - loss: 2.8184 - accuracy: 0.2946 - val_loss: 2.8270 - val_accuracy: 0.3166
Epoch 7/10
103/103 [==============================] - 0s 1ms/step - loss: 2.8155 - accuracy: 0.2953 - val_loss: 2.8239 - val_accuracy: 0.3166
Epoch 

In [14]:
# from sklearn.metrics import classification_report, confusion_matrix

# y_predict=model.predict(validation_data)
# y_predict= np.argmax(y_predict, axis=1)
# #print(y_predict)
# y_true=np.argmax(validation_labels, axis=1)
# #print(y_true)
# print(classification_report(y_predict,y_true))
# y_predict=list(y_predict)
# y_true=list(y_true)
# with open("Mobilenet_DataAug.txt","w+") as f:
#     f.write(str([y_predict,y_true]))

In [15]:
indexes=[]
for x in range(len(y_true)):
    if y_true[x]!=y_predict[x]:
    #print(y_true[x], y_predict[x])
        indexes.append(x)
print(len(indexes))



NameError: name 'y_true' is not defined

In [ ]:
# import shutil
# dir_hapus="/media/hades/DISK 0 - Drive 3/Fish/Test7/Test/"
# for i in indexes[0:-20]:
#     print(list_file[i])
#     shutil.move(dir_hapus+list_file[i],"/media/hades/DISK 0 - Drive 3/Fish/Test7/Test_Backup/")
    

In [ ]:
#Graphing our training and validation
#print(history.history)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy') 
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss') 
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
hasil=os.listdir("/media/hades/DISK 0 - Drive 3/Fish/Test7/Test/")

consumable=pd.read_csv("Commercial.txt",delimiter=r"\t+")
consumable2=pd.read_csv("Game_Fishes.txt",delimiter=r"\t+")

unconsumable=pd.read_csv("Dangerous.txt",delimiter=r"\t+")


unconsumable_final=[]
consumable_final=[]
consumable2_final=[]

for x in consumable['Species']:
    x=x.replace(' ','_')
    consumable_final.append(x)
for x in unconsumable['Species']:
    x=x.replace(' ','_')
    unconsumable_final.append(x)
for x in consumable2['Species']:
    x=x.replace(' ','_')
    consumable2_final.append(x)

In [ ]:
def Sort_Tuple(tup):  
  
    # reverse = None (Sorts in Ascending order)  
    # key is set to sort using second element of  
    # sublist lambda has been used  
    tup.sort(key = lambda x: x[2],reverse=True)  
    return tup  

In [ ]:
def read_image(file_path):
   #print('[INFO] loading and preprocessing image…') 
   image = load_img(file_path, target_size=(224, 224)) 
   image = img_to_array(image) 
   image = np.expand_dims(image, axis=0)
   image /= 255. 
   return image
def test_single_image(path):
  animals = os.listdir('/media/hades/DISK 0 - Drive 3/Fish/Test7/Train')
  animals=sorted(animals)
  images = read_image(path)
  time.sleep(.5)
  bt_prediction = vgg16.predict(images) 
  preds = model.predict_proba(bt_prediction)
  #print("preds:",preds)
  finalpercent=Sort_Tuple(list(zip(range(0,667), animals , preds[0])))
  #print(finalpercent)
  #for idx, animal, x in zip(range(0,10), animals , preds[0]):
   #print('ID: {}, Label: {} {}%'.format(idx, animal, round(x*100,2) ))
    
  #print('Final Decision:')
  time.sleep(.5)
  for x in range(3):
   #print('.'*(x+1))
   time.sleep(.2)
  class_predicted = model.predict_classes(bt_prediction)
  class_dictionary = generator_top.class_indices 
  inv_map = {v: k for k, v in class_dictionary.items()} 
  #print("PRED:",class_dictionary.items())
  finalpred='ID: {}, Label: {}'.format(class_predicted[0],  inv_map[class_predicted[0]])
  finalpred2=[]
  for x,y in enumerate(finalpercent[:5]):
        finalpred2.append('ID: {}, Label: {} {}%'.format(y[0],y[1],y[2]))
  #print(finalpred2)
  #print('ID: {}, Label: {}'.format(class_predicted[0],  inv_map[class_predicted[0]])) 
  return finalpred,finalpred2,finalpercent

path = '/media/hades/DISK 0 - Drive 3/Fish/Test7/Test/Naso/Naso_unicornis_0060.jpg'


In [ ]:
def get_data(species):
    nama_species=species[0][1].replace("_",' ')
    print("namaSP:",species[0][1])
    pd.set_option('display.max_rows', unconsumable.shape[0]+1)
    print(unconsumable)
    if species[0][1] in consumable_final:
        row=consumable.loc[consumable['Species'] == nama_species]
        row_list=[]
        
        print("Tes1:",row['Species'])
        row_list.append("CONSUMABLE")
        row_list.append("Species Name: %s" %row['Species'].values[0])
        row_list.append("Order: %s" %row['Order'].values[0])
        row_list.append("Family: %s" %row['Family'].values[0])
        row_list.append("Foreign Name: %s" %row['FishBase name'].values[0])
        row_list.append("Local Name: %s" %row['Name'].values[0])
        row_list.append("Description: %s" %row['Use'].values[0])
    elif species[0][1] in consumable2_final:
        row=consumable2.loc[consumable2['Species'] == nama_species]
        row_list=[]
        
        print("Tes2:",row['Species'])
        row_list.append("CONSUMABLE")
        row_list.append("Species Name: %s" %row['Species'].values[0])
        row_list.append("Order: %s" %row['Order'].values[0])
        row_list.append("Family: %s" %row['Family'].values[0])
        row_list.append("Foreign Name: %s" %row['FishBase name'].values[0])
        row_list.append("Local Name: %s" %row['Name'].values[0])
        row_list.append("Description: %s" %row['No'].values[0])
    else:
        print("ASDDDDDDDDDDDDDDDD",nama_species)
        row=unconsumable.loc[unconsumable['Species'] == nama_species]
        row_list=[]
        
        print("Tes3:",row['Species'])
        row_list.append("UNCONSUMABLE")
        row_list.append("Species Name: %s" %row['Species'].values[0])
        row_list.append("Order: %s" %row['Order'].values[0])
        row_list.append("Family: %s" %row['Family'].values[0])
        row_list.append("Foreign Name: %s" %row['FishBase name'].values[0])
        row_list.append("Local Name: %s" %row['Name'].values[0])
        row_list.append("Description: %s" %row['Danger'].values[0])
        
    return row_list

In [ ]:
import matplotlib.image as mpimg
%matplotlib tk

img = mpimg.imread(path)
plt.imshow(img)
a,b,c=test_single_image(path)
print("HH",b)
#print(get_data(c))
textstr = '\n'.join((
    r'Classification Result',
        r'',
    r'',
    r'Final Result:',
        r'',
        r'%s'%a,
        r'',
        r'DEBUG: ',
        r'',
        r'%s'%b[0],
        r'%s'%b[1],
        r'%s'%b[2],
        r'%s'%b[3],
        r'%s'%b[4]
    ))
d=get_data(c)
#print(get_data(c))
#print(get_data(c))
textstr2 = '\n'.join((
        r'%s'%d[0],
        r'Details',
        r'',
        r'%s'%d[1],
        r'%s'%d[2],
        r'%s'%d[3],
        r'%s'%d[4],
        r'%s'%d[5],
        r'%s'%d[6],
        
    ))
plt.xlabel(' ')
plt.ylabel(' ')






props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.subplots_adjust(left=0.35)

plt.text(0.05, 0.95, textstr, transform=plt.gcf().transFigure, fontsize=10,
        verticalalignment='top', bbox=props)

plt.text(0.05, 0.42, textstr2, transform=plt.gcf().transFigure, fontsize=10,
        verticalalignment='top', bbox=props)

plt.show()


